## Research Assistant Agent

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
from dotenv import load_dotenv

from agentic_newsroom.schemas.models import StoryBrief, ResearchPackage


# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)

print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")

## Configuration    

In [ ]:
slug = "bounty-mutiny-1789-feature"

from agentic_newsroom.llm.openai import get_mini_model

mini_model = get_mini_model()
config = {"configurable": {"model": mini_model}}

## Nodes

In [ ]:
from agentic_newsroom.agents.research_assistant import (
    generate_queries_node,
    search_node,
    curate_node,
    extract_analyze_node,
    finalize_research_node
)

story_brief = StoryBrief.load(slug=slug)


In [ ]:
# Call the first tool node

research_state = {
    "story_brief": story_brief
}

# 2. Generate Queries
update1 = generate_queries_node(research_state, config=config)
research_state.update(update1) # MERGE UPDATE


In [ ]:
queries = research_state["queries"]
print(queries)

In [ ]:
# Call the second tool node

# 3. Search
update2 = search_node(research_state) # state has story_brief AND queries now
research_state.update(update2) # MERGE UPDATE


In [ ]:
raw_search_results = research_state["raw_search_results"]
print(raw_search_results[0])


In [ ]:
# 4. Curate
update3 = curate_node(research_state, config=config) # state has story_brief, queries, AND raw_results
research_state.update(update3)

In [ ]:
# 5. Extract and Analyze
update4 = extract_analyze_node(research_state, config=config)
research_state.update(update4)

In [ ]:
# 6. Finalize

update5 = finalize_research_node(research_state) # No config needed here
research_state.update(update5)

print(research_state["research_package"])

## Subgraph

In [ ]:
from agentic_newsroom.agents.research_assistant import build_research_assistant_graph

graph = build_research_assistant_graph()

# Visualize the graph
print(graph.get_graph().draw_ascii())

## Run the graph

In [ ]:
research_state = {
    "story_brief": story_brief
}

result = graph.invoke(research_state, config)

## Load the Serialized Research Package

In [ ]:
from IPython.display import display, Markdown

research_packge = ResearchPackage.load(slug=slug)

display(Markdown(research_packge.to_markdown()))